In [1]:
# just here
%load_ext autoreload
%autoreload 2

In [2]:
# imports
from copy import copy
import numpy as np
import pandas as pd
import pynapple as nap
import matplotlib.pyplot as plt
import iblphotometry.loaders as loaders
import iblphotometry.plots as plots

In [ ]:
# ONE instantiation
from one.api import ONE
one = ONE(base_url="https://alyx.internationalbrainlab.org", mode='remote')

In [ ]:
# load data via data_loader
eids = one.search(dataset='photometry.signal.pqt', lab='wittenlab')
eid = eids[2]

data_loader = loaders.PhotometryLoader(one)
raw_photometry = data_loader.load_photometry_data(eid, signal='GCaMP')
trials = data_loader.load_trials_table(eid)

In [ ]:
from iblphotometry.outlier_detection import remove_spikes
from iblphotometry.bleach_corrections import lowpass_bleachcorrect
from iblphotometry.sliding_operations import sliding_mad
from iblphotometry.pipelines import run_pipeline

pipeline = [
      (remove_spikes, dict(sd=5)),
      (lowpass_bleachcorrect, dict(correction_method='subtract-divide',
                                   filter_params=dict(N=3, Wn=0.01, btype='lowpass'))),
      (sliding_mad, dict(w_len=120,overlap=90)),
]

# run pipeline
photometry = run_pipeline(raw_photometry, pipeline, on_columns=True)
data_loader.get_mappable(eid)
# photometry


In [ ]:
# view a short snippet
interval = nap.IntervalSet(start=10, end=20)
plots.plot_TsdFrame(photometry.restrict(interval))

In [ ]:
# plot psths
region = 'Region0G'
plots.plot_psth(photometry[region], trials, event='feedback_times', split_by='choice')